https://www.kobis.or.kr/kobis/business/main/main.do

In [1]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
from urllib.parse import quote

import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from datetime import datetime, timedelta

In [36]:
movie_df = pd.read_csv("(완)[KOBIS] 박스오피스_일별(2003~2022)_장르_수정.csv")

In [17]:
genre = "액션"
genre_movies = movie_df[movie_df['장르'].str.contains(genre)].groupby(["영화명","개봉일"])['누적관객수'].max().sort_values(ascending=False).to_frame()
genre_movies = genre_movies[genre_movies['누적관객수']>1000000].reset_index()
genre_movies["영화명"]

0               명량
1       어벤져스: 엔드게임
2              베테랑
3              아바타
4              도둑들
          ...     
324          본 레거시
325     007 카지노 로얄
326        밴티지 포인트
327    존 윅 3: 파라벨룸
328        스카이스크래퍼
Name: 영화명, Length: 329, dtype: object

In [18]:
movie_name = genre_movies['영화명'][0]
# movie_name = '스즈메의 문단속'
keyword = "영화 "+ str(movie_name) +" 평점"
keyword

'영화 명량 평점'

In [19]:
# 셀레니움으로 네이버 창 열기
driver = webdriver.Chrome()
time.sleep(2)
url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%ED%8F%89%EC%A0%90&oquery=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%EC%A0%95%EB%B3%B4&tqi=iaz3Ysp0YiRssLJ4UA4ssssss8Z-510880"
driver.get(url)
time.sleep(3)

In [20]:
# 검색어 입력 및 검색
driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').clear()
driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').send_keys(keyword)
driver.find_element(By.CSS_SELECTOR,'button.bt_search').click()
time.sleep(2)

In [21]:
# 공감순 선택
driver.find_element(By.CSS_SELECTOR, 'li.tab._sorting_select_order._trigger span.ico_check_wrap').click()
time.sleep(1)
driver.find_element(By.CSS_SELECTOR, 'div.cm_filter_area._root._sorting_select_order ul li:nth-child(1) a').click()
time.sleep(1)

In [201]:
# 리뷰 스크롤 내리기
driver.find_element(By.CSS_SELECTOR,'div.lego_review_list._scroller').click()

# 10번
for i in range(0,3):
    driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
    time.sleep(0.5)

In [22]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [180]:
제목 = soup.select_one('div._au_movie_content_wrap span strong._text').text
제목

'부러진 화살'

In [181]:
전체 = soup.select_one('span.area_star_number').text
참여자수 = soup.select_one('span.area_people').text.split('명 참여')[0]
남자 = soup.select_one('div.area_card_male span.area_star_number').text
여자 = soup.select_one('div.area_card_female span.area_star_number').text
print(전체, 참여자수, 남자, 여자)

8.91 7,378 8.68 9.30


In [182]:
_10대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(1) span.this_text_num').text
_20대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(2) span.this_text_num').text
_30대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(3) span.this_text_num').text
_40대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(4) span.this_text_num').text
_50대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(5) span.this_text_num').text
print(_10대, _20대, _30대, _40대, _50대)

9.18 8.77 8.88 9.01 8.55


In [280]:
lis = soup.select('div.lego_review_list._scroller ul._item_wrapper li')
li= lis[0]
관람객별점 = li.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
관람객리뷰 = li.select_one('div.area_text_expand span.desc._text').text.strip()
print(관람객별점, 관람객리뷰)

10 가볍게 생각 없이 즐길수 있는 영화 잘봤습니다


In [196]:
# 평론가 클릭 및 스크롤 내리기
driver.find_element(By.CSS_SELECTOR,'div.cm_tap_area._tab_wrap ul li:nth-child(2) a span').click()
driver.find_element(By.CSS_SELECTOR,'div.lego_critic_outer._scroller').click()

for i in range(0,3):
    driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
    time.sleep(0.5)

In [85]:
lis = soup.select('div.lego_critic_outer._scroller ul.area_ulist li')
li= lis[0]
평론가별점 = li.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
평론가리뷰 = li.select_one('div.area_text_expand').text.strip()
print(평론가별점, 평론가리뷰)

7.0 해체 위기에 놓인 마약 수사반이 잠복근무를 위해 치킨집을 인수한다. 공무원이 자영업자가 되면... 펼쳐보기


In [ ]:
#main_pack > div.sc_new.cs_common_module.case_empasis.color_5._au_movie_content_wrap > div.cm_content_wrap > div.cm_content_area._cm_content_area_rating > div > div:nth-child(3) > div > ul > li:nth-child(6) > div.area_text_expand > span

In [116]:
for num in range(1,len(driver.find_elements(By.CSS_SELECTOR,'div.lego_critic_outer._scroller ul.area_ulist li'))+1):
    try: 
        driver.find_element(By.CSS_SELECTOR,f'div.lego_critic_outer._scroller ul.area_ulist li:nth-child({num}) div.area_text_expand button').click()
        time.sleep(1)
    except:
        pass

In [74]:
# 기본정보 클릭 러닝타임 추출
driver.find_element(By.CSS_SELECTOR,'div.sub_tap_area._scrolling_wrapper_common_tab._scroll_mover ul li:nth-child(2)').click()
time.sleep(2)

divs = driver.find_elements(By.CSS_SELECTOR,'div.cm_content_area._cm_content_area_info div.detail_info dl div')

for div in divs:
    if '러닝타임' in div.text:
        러닝타임 = div.text.split()[1]
    elif '개봉' in div.text:
        개봉 = datetime.strptime(div.text.split()[1], '%Y.%m.%d.')
print(러닝타임, 개봉)

128분 2014-07-30 00:00:00


In [22]:
# 역대 영화 200개 데이터 불러오기
movie_df = pd.read_csv("(완)[KOBIS] 박스오피스_일별(2003~2022)_장르_수정.csv")

# 장르별 영화 크롤링
genre = "액션"
genre_movies = movie_df[movie_df['장르'].str.contains(genre)].groupby(["영화명","개봉일"])['누적관객수'].max().sort_values(ascending=False).to_frame()
genre_movies = genre_movies[genre_movies['누적관객수']>1000000].reset_index()
genre_movies

,영화명,개봉일,누적관객수
0,명량,2014-07-30,17615919
1,어벤져스: 엔드게임,2019-04-24,13977602
2,베테랑,2015-08-05,13414484
3,아바타,2009-12-17,13338863
4,도둑들,2012-07-25,12984701
...,...,...,...
324,본 레거시,2012-09-06,1015832
325,007 카지노 로얄,2006-12-21,1012087
326,밴티지 포인트,2008-02-28,1007729
327,존 윅 3: 파라벨룸,2019-06-26,1007383


# 영화정보, 평점, 리뷰 크롤링

In [37]:
# 역대 영화 200개 데이터 불러오기
movie_df = pd.read_csv("(완)[KOBIS] 박스오피스_일별(2003~2022)_장르_수정.csv")

# 장르별 영화 크롤링
genre_movies = 
genre_movies = genre_movies[genre_movies['누적관객수']>1000000].reset_index()
genre_movies["영화명"]

0               명량
1       어벤져스: 엔드게임
2              베테랑
3              아바타
4              도둑들
          ...     
324          본 레거시
325     007 카지노 로얄
326        밴티지 포인트
327    존 윅 3: 파라벨룸
328        스카이스크래퍼
Name: 영화명, Length: 329, dtype: object

In [38]:
movie_df.columns

Index(['순위', '영화명', '개봉일', '등급', '장르', '대표국적', '국적', '제작사', '배급사', '감독', '배우 ',
       '기준일', '매출액', '누적매출액', '관객수', '누적관객수', '스크린수', '상영횟수'],
      dtype='object')

In [45]:
movie_df.groupby(["영화명","개봉일","장르","대표국적"])["누적관객수","누적매출액"].max().reset_index().sort_values(['누적매출액'],ascending=False)

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_12940\1873791325.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  movie_df.groupby(["영화명","개봉일","장르","대표국적"])["누적관객수","누적매출액"].max().reset_index()


,영화명,개봉일,장르,대표국적,누적관객수,누적매출액
0,#살아있다,2020-06-24,드라마,한국,1903992,15968219900
1,#아이엠히어,2021-01-14,"멜로/로맨스,코미디",프랑스,16404,136875710
2,#위왓치유,2021-06-03,다큐멘터리,기타,1908,16759950
3,...ing,2003-11-28,"드라마,멜로/로맨스",한국,34308,217487000
4,0.0MHz,2019-05-29,공포(호러),한국,137294,1088258860
...,...,...,...,...,...,...
18132,"힘내세요, 병헌씨",2013-06-27,"코미디,드라마",한국,3980,28862600
18133,"힘을 내요, 미스터 리",2019-09-11,"코미디,드라마",한국,1181960,9930518906
18134,힙스터: 안개의 덫,2015-10-21,"멜로/로맨스,드라마",한국,1,5000
18135,힛쳐,2007-08-01,"액션,범죄,스릴러",미국,23025,153080000


In [59]:
movie_df.groupby(["영화명","개봉일","장르","대표국적"])["누적관객수","누적매출액"].max().sort_values(['누적매출액'],ascending=False).reset_index()

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_12940\1767948804.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  movie_df.groupby(["영화명","개봉일","장르","대표국적"])["누적관객수","누적매출액"].max().sort_values(['누적매출액'],ascending=False).reset_index()


,영화명,개봉일,장르,대표국적,누적관객수,누적매출액
0,극한직업,2019-01-23,코미디,한국,16266338,139655543516
1,명량,2014-07-30,"사극,액션",한국,17615919,135758658810
2,범죄도시2,2022-05-18,"범죄,액션",한국,12693302,131297440478
3,아바타,2009-12-17,"SF,액션,어드벤처",미국,13338863,125304346000
4,어벤져스: 엔드게임,2019-04-24,"액션,SF",미국,13977602,122492181020
...,...,...,...,...,...,...
18132,나무들 비탈에 서다,1968-04-13,드라마,한국,0,0
18133,장군의 아들 3,1992-07-11,"액션,드라마",한국,0,0
18134,결혼이야기,1992-07-04,"드라마,멜로/로맨스,코미디",한국,0,0
18135,붉은 유성,2008-12-22,드라마,일본,0,0


In [64]:
# 역대 영화 200개 데이터 불러오기
movie_df = pd.read_csv("(완)[KOBIS] 박스오피스_일별(2003~2022)_장르_수정.csv")

# 장르별 영화 크롤링
genre_movies = movie_df.groupby(["영화명","개봉일","장르","대표국적"])["누적관객수","누적매출액"].max().sort_values(['누적매출액'],ascending=False)
genre_movies = genre_movies[genre_movies["누적관객수"]>1000000].reset_index()

# 빈 데이터프레임 만들기
df1 = pd.DataFrame(columns = ['영화명', '개봉일', '장르','대표국적','누적관객수','누적매출액',
                              '제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'])
df7 = pd.DataFrame(columns = ['구분','별점','리뷰','제목','개봉','영화명','개봉일'])

# 실패시 담을 리스트 만들기
fail_list=[]

# 셀레니움으로 네이버 영화 검색 창 열기
driver = webdriver.Chrome()
time.sleep(2)
url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%ED%8F%89%EC%A0%90&oquery=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%ED%8F%89%EC%A0%90&tqi=iaXngsp0Jywssd6FGoKssssstH0-119566"
driver.get(url)
time.sleep(3)

### 크롤링 ###

for i in tqdm(range(0,len(genre_movies['영화명']))):
# for i in tqdm(range(0,2)):

    # 장르 df의 i번째 인덱스의 기본 정보 추출 (시리즈형태와 데이터프레임 형태)
    info = genre_movies.iloc[i]
    info_df = genre_movies.iloc[i:i+1]
    
    try: 
        # 검색어 설정
        keyword = "영화 "+ str(info['영화명']) +" 평점"

        # 검색어 입력 및 검색
        driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').clear()
        driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').send_keys(keyword)
        driver.find_element(By.CSS_SELECTOR,'button.bt_search').click()
        time.sleep(3)

        # 리뷰 공감순 선택
        driver.find_element(By.CSS_SELECTOR, 'li.tab._sorting_select_order._trigger span.ico_check_wrap').click()
        time.sleep(2)
        driver.find_element(By.CSS_SELECTOR, 'div.cm_filter_area._root._sorting_select_order ul li:nth-child(1) a').click()
        time.sleep(2)

        # 관람객 리뷰 스크롤 내리기
        driver.find_element(By.CSS_SELECTOR,'div.lego_review_list._scroller').click()
        time.sleep(1)

        for j in range(0,5):
            driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
            time.sleep(1)

        # 평론가 클릭 및 스크롤 내리기
        driver.find_element(By.CSS_SELECTOR,'div.cm_tap_area._tab_wrap ul li:nth-child(2) a span').click()
        driver.find_element(By.CSS_SELECTOR,'div.lego_critic_outer._scroller').click()
        time.sleep(1)

        for k in range(0,5):
            driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
            time.sleep(1)
        
        # 평론가 리뷰 중 가려진 텍스트 펼치기
        for num in range(1,len(driver.find_elements(By.CSS_SELECTOR,'div.lego_critic_outer._scroller ul.area_ulist li'))+1):
            try: 
                driver.find_element(By.CSS_SELECTOR,f'div.lego_critic_outer._scroller ul.area_ulist li:nth-child({num}) div.area_text_expand button').click()
                time.sleep(1)
            except:
                pass 
        
        # soup에 html 담기
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        # 변수 초기화
        제목 = "-"
        전체 = "-"
        참여자수 = "-"
        남자 = "-"
        여자 = "-"

        _10대 = "-"
        _20대 = "-"
        _30대 = "-"
        _40대 = "-"
        _50대 = "-"

        
        # 변수에 해당되는 값 추출해서 담기
        제목 = soup.select_one('div._au_movie_content_wrap span strong._text').text
        전체 = soup.select_one('span.area_star_number').text
        참여자수 = soup.select_one('span.area_people').text.split('명 참여')[0]
        남자 = soup.select_one('div.area_card_male span.area_star_number').text
        여자 = soup.select_one('div.area_card_female span.area_star_number').text

        _10대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(1) span.this_text_num').text
        _20대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(2) span.this_text_num').text
        _30대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(3) span.this_text_num').text
        _40대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(4) span.this_text_num').text
        _50대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(5) span.this_text_num').text

        # 빈 리스트 만들기
        df4 = []
        df5 = []

        # 관람객 리뷰
        lis1 = soup.select('div.lego_review_list._scroller ul._item_wrapper li')
        
        관람객별점 = '-'
        관람객리뷰 = '-'
        
        for li1 in lis1:
            관람객별점 = li1.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
            관람객리뷰 = li1.select_one('div.area_text_expand span.desc._text').text.strip()
            df4.append(['관람객', 관람객별점, 관람객리뷰])

        # 평론가 리뷰
        lis2 = soup.select('div.lego_critic_outer._scroller ul.area_ulist li')
        
        평론가별점 = '-'
        평론가리뷰 = '-'
        
        for li2 in lis2:
            평론가별점 = li2.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
            평론가리뷰 = li2.select_one('div.area_text_expand').text.strip()
            df5.append(['평론가', 평론가별점, 평론가리뷰])

        # 기본정보 클릭 개봉일, 러닝타임 추출
        driver.find_element(By.CSS_SELECTOR,'div.sub_tap_area._scrolling_wrapper_common_tab._scroll_mover ul li:nth-child(2)').click()
        time.sleep(3)

        sub_soup = BeautifulSoup(driver.page_source, "html.parser")
    
        divs = sub_soup.select('div.cm_content_area._cm_content_area_info div.detail_info dl div')
        
        러닝타임 = '-'
        개봉 = '-'

        for div in divs:
            if '러닝타임' in div.text:
                러닝타임 = div.text.split()[1]
            elif '개봉' in div.text:
                개봉 = datetime.strptime(div.text.split()[1], '%Y.%m.%d.')

         
        # [평점]
        # 역대 기본 정보와 크롤링한 정보 합치기
        df2_list = [[제목, 러닝타임, 개봉, 전체, 참여자수, 남자, 여자, _10대, _20대, _30대, _40대, _50대]]
        df2 = pd.DataFrame(df2_list, columns = ['제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'], index=[i])
        df3 = pd.concat([info_df, df2], axis=1)
        
        # 최종 데이터프레임에 추가하기
        df1 = pd.concat([df1, df3])
        
        # [리뷰]
        # 역대 기본 정보 영화명, 개봉일과 크롤링한 정보 합치기
        df4 = pd.DataFrame(df4, columns = ['구분','별점','리뷰'])
        df4['제목'] = 제목
        df4['개봉'] = 개봉
        df4['영화명'] = info['영화명']
        df4['개봉일'] = info['개봉일']

        df5 = pd.DataFrame(df5, columns = ['구분','별점','리뷰'])
        df5['제목'] = 제목
        df5['개봉'] = 개봉
        df5['영화명'] = info['영화명']
        df5['개봉일'] = info['개봉일']
         
        df6 = pd.concat([df4,df5])
        df7 = pd.concat([df7,df6])
        
    # 실패시 fail_list에 영화명 개봉일 담기 
    except:
        print(info['영화명'])
        fail_list.append([info['영화명'],info['개봉일']])
        
# 최종 데이터 프레임 컬럼명 설정
best_movie_star = pd.DataFrame(df1, columns=['영화명', '개봉일', '장르','대표국적','누적관객수','누적매출액',
                                             '제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'])
best_movie_review = pd.DataFrame(df7, columns=['구분','별점','리뷰','제목','개봉','영화명','개봉일'])
best_movie_review = best_movie_review[['영화명','개봉일', '제목','개봉', '구분','별점','리뷰']]
fail_list_df = pd.DataFrame(fail_list, columns=['영화명','개봉일'])

# 엑셀 및 csv 파일로 저장
best_movie_star.to_excel('[크롤링] 관객_백만명이상_네이버_영화_평점.xlsx', index=False)
best_movie_review.to_excel('[크롤링] 관객_백만명이상_네이버_영화_리뷰.xlsx', index=False)
fail_list_df.to_csv('[크롤링] 관객_백만명이상_네이버_영화_실패.csv', index=False)

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_12940\638496975.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  genre_movies = movie_df.groupby(["영화명","개봉일","장르","대표국적"])["누적관객수","누적매출액"].max().sort_values(['누적매출액'],ascending=False).reset_index()


  0%|          | 0/776 [00:00<?, ?it/s]

극한직업
명량
범죄도시2
아바타
어벤져스: 엔드게임
신과함께-죄와 벌
겨울왕국 2
국제시장
알라딘
베테랑
신과함께-인과 연
어벤져스: 인피니티 워
암살
택시운전사
도둑들
부산행
7번방의 선물
아바타: 물의 길
광해, 왕이 된 남자
어벤져스: 에이지 오브 울트론
탑건: 매버릭
기생충
보헤미안 랩소디
인터스텔라
변호인
겨울왕국
해운대
엑시트
검사외전
스파이더맨: 노 웨이 홈
트랜스포머 3
한산: 용의 출현
캡틴 아메리카: 시빌 워
공조2: 인터내셔날
아이언맨 3
백두산
스파이더맨: 파 프롬 홈
설국열차
괴물
해적: 바다로 간 산적
왕의 남자
관상
공조
수상한 그녀
닥터 스트레인지: 대혼돈의 멀티버스
밀정
히말라야
어벤져스
스파이더맨: 홈 커밍
1987
마스터
국가대표
터널
미션임파서블:고스트프로토콜
내부자들
럭키
범죄도시
미션 임파서블: 폴아웃
곡성
최종병기 활
인천상륙작전
써니
과속스캔들
베를린
캡틴 마블
트랜스포머: 패자의 역습
군함도
킹스맨 : 시크릿 에이전트
쥬라기 월드: 폴른 킹덤
디워
사도
은밀하게 위대하게
미션 임파서블: 로그네이션
다크 나이트 라이즈
쥬라기 월드
트랜스포머
닥터 스트레인지
앤트맨과 와스프
아저씨
늑대소년
안시성
블랙 팬서
연평해전
조커
인셉션
헌트
덕혜옹주
청년경찰
완벽한 타인
쿵푸팬더 2
아쿠아맨
화려한 휴가
전우치
트랜스포머: 사라진 시대
좋은 놈, 나쁜 놈, 이상한 놈
더 킹
독전
레미제라블
공작
검은 사제들
미녀와 야수
라이온 킹
남산의 부장들
어메이징 스파이더맨
킹스맨: 골든 서클
봉오동 전투
웰컴 투 동막골
의형제
더 테러 라이브
마션
숨바꼭질
나쁜 녀석들: 더 무비
감시자들
신비한 동물사전
토르: 라그나로크
2012
엣지 오브 투모로우
다만 악에서 구하소서
월드 워 Z
완득이
인사이드 아웃
타워
님아, 그 강을 건너지 마오
주토피아
군도: 민란의 시대
범죄와의 전쟁: 나쁜놈들 전성시대
미녀는 괴로워
타짜
판도라
조선명탐정 : 각시투구꽃의 비밀
도가니
강철비
아가씨
신세계
어메이징 스파이더맨 2
해리포터와 죽음의 성물2
엑

In [63]:
fail_list_df

,영화명,개봉일


In [163]:
best_movie_star.to_csv('[크롤링] 역대_네이버_영화_평점_수정.csv', index=False)
best_movie_review.to_csv('[크롤링] 역대_네이버_영화_리뷰_수정.csv', index=False)

In [28]:
fail_list = pd.read_csv('[크롤링] 역대_네이버_영화_실패_수정.csv')
fail_list = fail_list.drop(index=0)
fail_list.to_excel('[크롤링] 역대_네이버_영화_실패_수정.xlsx', index=False)

# fail_list - URL 기준 크롤링

In [3]:
# 역대 영화 200개 데이터 불러오기
movie_df = pd.read_csv("(완)[KOBIS] 박스오피스_일별(2003~2022)_장르_수정.csv")

# 장르별 영화 크롤링
genre_movies = movie_df.groupby(["영화명","개봉일","장르","대표국적"])["누적관객수","누적매출액"].max().sort_values(['누적매출액'],ascending=False)
genre_movies = genre_movies[genre_movies["누적관객수"]>1000000].reset_index()

# 빈 데이터프레임 만들기
df1 = pd.DataFrame(columns = ['영화명', '개봉일', '장르','대표국적','누적관객수','누적매출액',
                              '제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'])
df7 = pd.DataFrame(columns = ['구분','별점','리뷰','제목','개봉','영화명','개봉일'])

fail_list=[]

# 셀레니움으로 네이버 창 열기
driver = webdriver.Chrome()
time.sleep(2)
url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%ED%8F%89%EC%A0%90&oquery=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%ED%8F%89%EC%A0%90&tqi=iaXngsp0Jywssd6FGoKssssstH0-119566"
driver.get(url)
time.sleep(3)

# 실패 리스트에서 URL 추가한 파일 불러오기
fail_list_link = pd.read_excel('[크롤링] 관객_백만명이상_네이버_영화_실패.xlsx')
fail_list_link

for fail_index in tqdm(range(0,len(fail_list_link['영화명']))):
    
    movie_keyword = fail_list_link['영화명'][fail_index]

    i = genre_movies[genre_movies['영화명']==movie_keyword].index[0]
    info = genre_movies.iloc[i]
    info_df = genre_movies.iloc[i:i+1]

    try: 
        # 검색어 설정
        keyword = "영화 "+ str(movie_keyword) +" 평점"

        # 해당 영화 정보 링크 들어가기
        sub_url = fail_list_link['링크'][fail_index]
        driver.get(sub_url)
        time.sleep(3)

        # 리뷰 공감순 선택
        driver.find_element(By.CSS_SELECTOR, 'li.tab._sorting_select_order._trigger span.ico_check_wrap').click()
        time.sleep(2)
        driver.find_element(By.CSS_SELECTOR, 'div.cm_filter_area._root._sorting_select_order ul li:nth-child(1) a').click()
        time.sleep(2)
        
        # 리뷰 공감순 선택
        driver.find_element(By.CSS_SELECTOR, 'li.tab._sorting_select_order._trigger span.ico_check_wrap').click()
        time.sleep(2)
        driver.find_element(By.CSS_SELECTOR, 'div.cm_filter_area._root._sorting_select_order ul li:nth-child(1) a').click()
        time.sleep(2)

        # 관람객 리뷰 스크롤 내리기
        driver.find_element(By.CSS_SELECTOR,'div.lego_review_list._scroller').click()
        time.sleep(1)
        driver.find_element(By.CSS_SELECTOR,'div.lego_review_list._scroller').click()
        time.sleep(1)

        for j in range(0,2):
            driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
            time.sleep(1)

        # 평론가 클릭 및 스크롤 내리기
        driver.find_element(By.CSS_SELECTOR,'div.cm_tap_area._tab_wrap ul li:nth-child(2) a span').click()
        driver.find_element(By.CSS_SELECTOR,'div.lego_critic_outer._scroller').click()
        time.sleep(1)

        for k in range(0,2):
            driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
            time.sleep(1)
        
        # 평론가 리뷰 중 가려진 텍스트 펼치기
        for num in range(1,len(driver.find_elements(By.CSS_SELECTOR,'div.lego_critic_outer._scroller ul.area_ulist li'))+1):
            try: 
                driver.find_element(By.CSS_SELECTOR,f'div.lego_critic_outer._scroller ul.area_ulist li:nth-child({num}) div.area_text_expand button').click()
                time.sleep(1)
            except:
                pass 
        
        # soup에 html 담기
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        # 변수 초기화
        제목 = "-"
        전체 = "-"
        참여자수 = "-"
        남자 = "-"
        여자 = "-"

        _10대 = "-"
        _20대 = "-"
        _30대 = "-"
        _40대 = "-"
        _50대 = "-"

        
        # 변수에 해당되는 값 추출해서 담기
        제목 = soup.select_one('div._au_movie_content_wrap span strong._text').text
        전체 = soup.select_one('span.area_star_number').text
        참여자수 = soup.select_one('span.area_people').text.split('명 참여')[0]
        남자 = soup.select_one('div.area_card_male span.area_star_number').text
        여자 = soup.select_one('div.area_card_female span.area_star_number').text

        _10대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(1) span.this_text_num').text
        _20대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(2) span.this_text_num').text
        _30대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(3) span.this_text_num').text
        _40대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(4) span.this_text_num').text
        _50대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(5) span.this_text_num').text

        # 빈 리스트 만들기
        df4 = []
        df5 = []

        # 관람객 리뷰
        lis1 = soup.select('div.lego_review_list._scroller ul._item_wrapper li')
        
        관람객별점 = '-'
        관람객리뷰 = '-'
        
        for li1 in lis1:
            관람객별점 = li1.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
            관람객리뷰 = li1.select_one('div.area_text_expand span.desc._text').text.strip()
            df4.append(['관람객', 관람객별점, 관람객리뷰])

        # 평론가 리뷰
        lis2 = soup.select('div.lego_critic_outer._scroller ul.area_ulist li')
        
        평론가별점 = '-'
        평론가리뷰 = '-'
        
        for li2 in lis2:
            평론가별점 = li2.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
            평론가리뷰 = li2.select_one('div.area_text_expand').text.strip()
            df5.append(['평론가', 평론가별점, 평론가리뷰])

        # 기본정보 클릭 개봉일, 러닝타임 추출
        driver.find_element(By.CSS_SELECTOR,'div.sub_tap_area._scrolling_wrapper_common_tab._scroll_mover ul li:nth-child(2)').click()
        time.sleep(3)
        driver.find_element(By.CSS_SELECTOR,'div.sub_tap_area._scrolling_wrapper_common_tab._scroll_mover ul li:nth-child(2)').click()
        time.sleep(3)

        sub_soup = BeautifulSoup(driver.page_source, "html.parser")
    
        divs = sub_soup.select('div.cm_content_area._cm_content_area_info div.detail_info dl div')
        
        러닝타임 = '-'
        개봉 = '-'

        for div in divs:
            if '러닝타임' in div.text:
                러닝타임 = div.text.split()[1]
            elif '개봉' in div.text:
                개봉 = datetime.strptime(div.text.split()[1], '%Y.%m.%d.')

         
        # [평점]
        # 역대 기본 정보와 크롤링한 정보 합치기
        df2_list = [[제목, 러닝타임, 개봉, 전체, 참여자수, 남자, 여자, _10대, _20대, _30대, _40대, _50대]]
        df2 = pd.DataFrame(df2_list, columns = ['제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'], index=[i])
        df3 = pd.concat([info_df, df2], axis=1)
        
        # 최종 데이터프레임에 추가하기
        df1 = pd.concat([df1, df3])
        
        # [리뷰]
        # 역대 기본 정보 영화명, 개봉일과 크롤링한 정보 합치기
        df4 = pd.DataFrame(df4, columns = ['구분','별점','리뷰'])
        df4['제목'] = 제목
        df4['개봉'] = 개봉
        df4['영화명'] = info['영화명']
        df4['개봉일'] = info['개봉일']

        df5 = pd.DataFrame(df5, columns = ['구분','별점','리뷰'])
        df5['제목'] = 제목
        df5['개봉'] = 개봉
        df5['영화명'] = info['영화명']
        df5['개봉일'] = info['개봉일']
         
        df6 = pd.concat([df4,df5])
        df7 = pd.concat([df7,df6])
        
    # 실패시 fail_list에 영화명 개봉일 담기 
    except:
        print(info['영화명'])
        fail_list.append([info['영화명'],info['개봉일']])
        
# 최종 데이터 프레임 컬럼명 설정
best_movie_star = pd.DataFrame(df1, columns=['영화명', '개봉일', '장르','대표국적','누적관객수','누적매출액',
                                             '제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'])
best_movie_review = pd.DataFrame(df7, columns=['구분','별점','리뷰','제목','개봉','영화명','개봉일'])
best_movie_review = best_movie_review[['영화명','개봉일', '제목','개봉', '구분','별점','리뷰']]
fail_list_df = pd.DataFrame(fail_list, columns=['영화명','개봉일'])

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_6816\678229676.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  genre_movies = movie_df.groupby(["영화명","개봉일","장르","대표국적"])["누적관객수","누적매출액"].max().sort_values(['누적매출액'],ascending=False)


  0%|          | 0/140 [00:00<?, ?it/s]

워낭소리


In [6]:
best_movie_star_df = pd.read_excel('[크롤링] 관객_백만명이상_네이버_영화_평점.xlsx')
best_movie_review_df = pd.read_excel('[크롤링] 관객_백만명이상_네이버_영화_리뷰.xlsx')

In [7]:
best_movie_star_df = pd.concat([best_movie_star_df, best_movie_star])
best_movie_review_df = pd.concat([best_movie_review_df, best_movie_review])

In [8]:
best_movie_star_df.to_excel('[크롤링] 관객_백만명이상_네이버_영화_평점_1.xlsx', index=False)
best_movie_review_df.to_excel('[크롤링] 관객_백만명이상_네이버_영화_리뷰_1.xlsx', index=False)

In [67]:
best_movie_review[best_movie_review["개봉일"]!=best_movie_review['개봉']]

,영화명,개봉일,제목,개봉,구분,별점,리뷰


In [95]:
df = best_movie_review_df[best_movie_review_df['구분']=="관람객"]['영화명'].value_counts().to_frame()

In [100]:
df[df['영화명']<30]

,영화명
다크 나이트 라이즈,16
어벤져스,10
아바타,10


# 영화명 하나씩 가져오기

In [10]:
genre_movies[genre_movies['영화명']==movie_keyword].index[0]

320

In [12]:
fail_index

139

In [14]:
 fail_list_link[fail_list_link['영화명']==movie_keyword].index[0]

139

In [15]:
 fail_index = fail_list_link[fail_list_link['영화명']==movie_keyword].index[0]

20

In [25]:
# 역대 영화 200개 데이터 불러오기
movie_df = pd.read_csv("(완)[KOBIS] 박스오피스_일별(2003~2022)_장르_수정.csv")

# 장르별 영화 크롤링
genre_movies = movie_df.groupby(["영화명","개봉일","장르","대표국적"])["누적관객수","누적매출액"].max().sort_values(['누적매출액'],ascending=False)
genre_movies = genre_movies[genre_movies["누적관객수"]>1000000].reset_index()

# 빈 데이터프레임 만들기
df1 = pd.DataFrame(columns = ['영화명', '개봉일', '장르','대표국적','누적관객수','누적매출액',
                              '제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'])
df7 = pd.DataFrame(columns = ['구분','별점','리뷰','제목','개봉','영화명','개봉일'])

fail_list=[]

# 셀레니움으로 네이버 창 열기
driver = webdriver.Chrome()
time.sleep(2)
url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%ED%8F%89%EC%A0%90&oquery=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%ED%8F%89%EC%A0%90&tqi=iaXngsp0Jywssd6FGoKssssstH0-119566"
driver.get(url)
time.sleep(3)

# 실패 리스트에서 URL 추가한 파일 불러오기
fail_list_link = pd.read_excel('[크롤링] 관객_백만명이상_네이버_영화_실패.xlsx')
fail_list_link

# for fail_index in tqdm(range(0,len(fail_list_link['영화명']))):
    
movie_keyword = "워낭소리"

i = genre_movies[genre_movies['영화명']==movie_keyword].index[0]
info = genre_movies.iloc[i]
info_df = genre_movies.iloc[i:i+1]

try: 
    # 검색어 설정
    keyword = "영화 "+ str(movie_keyword) +" 평점"
    
    fail_index = fail_list_link[fail_list_link['영화명']==movie_keyword].index[0]
    
    # 해당 영화 정보 링크 들어가기
    sub_url = fail_list_link['링크'][fail_index]
    driver.get(sub_url)
    time.sleep(3)

    # 리뷰 공감순 선택
    driver.find_element(By.CSS_SELECTOR, 'li.tab._sorting_select_order._trigger span.ico_check_wrap').click()
    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR, 'div.cm_filter_area._root._sorting_select_order ul li:nth-child(1) a').click()
    time.sleep(2)

    # 리뷰 공감순 선택
    driver.find_element(By.CSS_SELECTOR, 'li.tab._sorting_select_order._trigger span.ico_check_wrap').click()
    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR, 'div.cm_filter_area._root._sorting_select_order ul li:nth-child(1) a').click()
    time.sleep(2)

    # 관람객 리뷰 스크롤 내리기
    driver.find_element(By.CSS_SELECTOR,'div.lego_review_list._scroller').click()
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR,'div.lego_review_list._scroller').click()
    time.sleep(1)

    for j in range(0,2):
        driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
        time.sleep(1)

    # 평론가 클릭 및 스크롤 내리기
    driver.find_element(By.CSS_SELECTOR,'div.cm_tap_area._tab_wrap ul li:nth-child(2) a span').click()
    driver.find_element(By.CSS_SELECTOR,'div.lego_critic_outer._scroller').click()
    time.sleep(1)

    for k in range(0,2):
        driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
        time.sleep(1)

    # 평론가 리뷰 중 가려진 텍스트 펼치기
    for num in range(1,len(driver.find_elements(By.CSS_SELECTOR,'div.lego_critic_outer._scroller ul.area_ulist li'))+1):
        try: 
            driver.find_element(By.CSS_SELECTOR,f'div.lego_critic_outer._scroller ul.area_ulist li:nth-child({num}) div.area_text_expand button').click()
            time.sleep(1)
        except:
            pass 

    # soup에 html 담기
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 변수 초기화
    제목 = "-"
    전체 = "-"
    참여자수 = "-"
    남자 = "-"
    여자 = "-"

    _10대 = "-"
    _20대 = "-"
    _30대 = "-"
    _40대 = "-"
    _50대 = "-"


    # 변수에 해당되는 값 추출해서 담기
    제목 = soup.select_one('div._au_movie_content_wrap span strong._text').text
    전체 = soup.select_one('span.area_star_number').text
    참여자수 = soup.select_one('span.area_people').text.split('명 참여')[0]
    남자 = soup.select_one('div.area_card_male span.area_star_number').text
    여자 = soup.select_one('div.area_card_female span.area_star_number').text

    _10대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(1) span.this_text_num').text
    _20대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(2) span.this_text_num').text
    _30대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(3) span.this_text_num').text
    _40대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(4) span.this_text_num').text
    _50대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(5) span.this_text_num').text

    # 빈 리스트 만들기
    df4 = []
    df5 = []

    # 관람객 리뷰
    lis1 = soup.select('div.lego_review_list._scroller ul._item_wrapper li')

    관람객별점 = '-'
    관람객리뷰 = '-'

    for li1 in lis1:
        관람객별점 = li1.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
        관람객리뷰 = li1.select_one('div.area_text_expand span.desc._text').text.strip()
        df4.append(['관람객', 관람객별점, 관람객리뷰])

    # 평론가 리뷰
    lis2 = soup.select('div.lego_critic_outer._scroller ul.area_ulist li')

    평론가별점 = '-'
    평론가리뷰 = '-'

    for li2 in lis2:
        try:
            평론가별점 = li2.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
            평론가리뷰 = li2.select_one('div.area_text_expand').text.strip()
            df5.append(['평론가', 평론가별점, 평론가리뷰])
        except:
            pass
        
    # 기본정보 클릭 개봉일, 러닝타임 추출
#     driver.find_element(By.CSS_SELECTOR,'div.sub_tap_area._scrolling_wrapper_common_tab._scroll_mover ul li:nth-child(2)').click()
#     time.sleep(3)


    url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EC%9B%8C%EB%82%AD%EC%86%8C%EB%A6%AC+%EA%B8%B0%EB%B3%B8%EC%A0%95%EB%B3%B4&oquery=%EC%9B%8C%EB%82%AD%EC%86%8C%EB%A6%AC+%EC%A0%95%EB%B3%B4&tqi=ibxRjdprvmZssO9m5Cwssssst0C-289320"
    driver.get(url)
    time.sleep(3)

    sub_soup = BeautifulSoup(driver.page_source, "html.parser")

    divs = sub_soup.select('div.cm_content_area._cm_content_area_info div.detail_info dl div')

    러닝타임 = '-'
    개봉 = '-'

    for div in divs:
        if '러닝타임' in div.text:
            러닝타임 = div.text.split()[1]
        elif '개봉' in div.text:
            개봉 = datetime.strptime(div.text.split()[1], '%Y.%m.%d.')


    # [평점]
    # 역대 기본 정보와 크롤링한 정보 합치기
    df2_list = [[제목, 러닝타임, 개봉, 전체, 참여자수, 남자, 여자, _10대, _20대, _30대, _40대, _50대]]
    df2 = pd.DataFrame(df2_list, columns = ['제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'], index=[i])
    df3 = pd.concat([info_df, df2], axis=1)

    # 최종 데이터프레임에 추가하기
    df1 = pd.concat([df1, df3])

    # [리뷰]
    # 역대 기본 정보 영화명, 개봉일과 크롤링한 정보 합치기
    df4 = pd.DataFrame(df4, columns = ['구분','별점','리뷰'])
    df4['제목'] = 제목
    df4['개봉'] = 개봉
    df4['영화명'] = info['영화명']
    df4['개봉일'] = info['개봉일']

    df5 = pd.DataFrame(df5, columns = ['구분','별점','리뷰'])
    df5['제목'] = 제목
    df5['개봉'] = 개봉
    df5['영화명'] = info['영화명']
    df5['개봉일'] = info['개봉일']

    df6 = pd.concat([df4,df5])
    df7 = pd.concat([df7,df6])

# 실패시 fail_list에 영화명 개봉일 담기 
except:
    print(info['영화명'])
    fail_list.append([info['영화명'],info['개봉일']])

# 최종 데이터 프레임 컬럼명 설정
best_movie_star = pd.DataFrame(df1, columns=['영화명', '개봉일', '장르','대표국적','누적관객수','누적매출액',
                                             '제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'])
best_movie_review = pd.DataFrame(df7, columns=['구분','별점','리뷰','제목','개봉','영화명','개봉일'])
best_movie_review = best_movie_review[['영화명','개봉일', '제목','개봉', '구분','별점','리뷰']]
fail_list_df = pd.DataFrame(fail_list, columns=['영화명','개봉일'])

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_6816\250202484.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  genre_movies = movie_df.groupby(["영화명","개봉일","장르","대표국적"])["누적관객수","누적매출액"].max().sort_values(['누적매출액'],ascending=False)


In [30]:
best_movie_review

,영화명,개봉일,제목,개봉,구분,별점,리뷰
0,워낭소리,2009-01-15,워낭소리,2009-01-15,관람객,10,워낭소리 할아버지 ㅜㅜ 하늘에 가셔서 소하고 행복하게 지내세요
1,워낭소리,2009-01-15,워낭소리,2009-01-15,관람객,10,인간의 삶은 연출이 필요없음. 일단 한번 보셈.
2,워낭소리,2009-01-15,워낭소리,2009-01-15,관람객,10,고인의명복을빕니다.
3,워낭소리,2009-01-15,워낭소리,2009-01-15,관람객,10,동물학대라고생각하지마라 소가 할아버지의 사랑과 정성아니면 어떻게 40년동안 살았겠냐
4,워낭소리,2009-01-15,워낭소리,2009-01-15,관람객,10,"나를 울린 영화,,, 내 인생관과 닮은 우정을 보여준 영화,,, 죽은 소를 기립니다,,,"
5,워낭소리,2009-01-15,워낭소리,2009-01-15,관람객,10,너무 짠하네요... 눈물이 주르륵
6,워낭소리,2009-01-15,워낭소리,2009-01-15,관람객,10,현시대의 수백억을 들여만든 한국영화가 이영화가 주는 감동의 반이라도 따라올수있는 영...
7,워낭소리,2009-01-15,워낭소리,2009-01-15,관람객,10,'소'처럼 순진하기만하고 개성없는 동물이 또 있을까? 주인공 '소'가 만약 주인공 ...
8,워낭소리,2009-01-15,워낭소리,2009-01-15,관람객,10,저 어릴때 워낭소리 봤었는데...그때는 어찌 지루하던지 하품만 나오던데 지금 보면 ...
9,워낭소리,2009-01-15,워낭소리,2009-01-15,관람객,10,농사를 위해 소를 부리는건지 아니면 소와같이 있고 싶어서 농사를 짓는건지 늙은 소와...


In [28]:
best_movie_star_df = pd.read_excel('[크롤링] 관객_백만명이상_네이버_영화_평점_1.xlsx')
best_movie_review_df = pd.read_excel('[크롤링] 관객_백만명이상_네이버_영화_리뷰_1.xlsx')

In [31]:
best_movie_star_df = pd.concat([best_movie_star_df, best_movie_star])
best_movie_review_df = pd.concat([best_movie_review_df, best_movie_review])

In [32]:
best_movie_star_df.to_excel('[크롤링] 관객_백만명이상_네이버_영화_평점_2.xlsx', index=False)
best_movie_review_df.to_excel('[크롤링] 관객_백만명이상_네이버_영화_리뷰_2.xlsx', index=False)

# 이상요인 판단

In [151]:
best_movie_star_df = pd.read_excel('[크롤링] 역대_네이버_영화_평점_최종_수정.xlsx')
best_movie_review_df = pd.read_excel('[크롤링] 역대_네이버_영화_리뷰_최종_수정.xlsx')

In [152]:
best_movie_review_df.groupby(['영화명', '구분'])['구분'].count().to_frame()

구분
영화명          구분     
1987         관람객  30
             평론가  10
2012         관람객  30
             평론가  10
7급 공무원       관람객  30
...               ..
혹성탈출: 반격의 서막 평론가  10
화려한 휴가       관람객  30
             평론가   8
히말라야         관람객  30
             평론가  10

[398 rows x 1 columns]

In [155]:
df = best_movie_review_df[best_movie_review_df['구분']=="관람객"]['영화명'].value_counts().to_frame()
df[df['영화명']<30]

,영화명
어벤져스,20
아바타,19
다크 나이트 라이즈,16


In [162]:
# 관람객 리뷰 별점
best_movie_review_df[best_movie_review_df['구분']=="관람객"]['별점'].value_counts().to_frame()

,별점
10.0,3548
9.0,695
8.0,555
1.0,374
7.0,237
6.0,198
5.0,116
4.0,97
2.0,78
3.0,67
